In [3]:
pip install pydub pretty_midi librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.7 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=fecc832b8b6ce3f32d8a91181ce04e0572d2d24a9a1e76f1641568932ab39217
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi


In [4]:

import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from pydub import AudioSegment
import pretty_midi
from collections import Counter


In [6]:


# 🎵 Convert MP3 to WAV
def convert_mp3_to_wav(mp3_path, wav_path="converted_audio.wav"):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format="wav")
    return wav_path

# 🎵 Load WAV and Extract Pitches
def extract_pitches(wav_path):
    y, sr = librosa.load(wav_path, sr=None)
    y_harmonic, _ = librosa.effects.hpss(y)  # Isolate melody
    pitches, magnitudes = librosa.piptrack(y=y_harmonic, sr=sr)

    time_stamps = np.linspace(0, len(y) / sr, num=pitches.shape[1])
    note_dict = {}

    for t in range(pitches.shape[1]):
        second = int(time_stamps[t])
        index = magnitudes[:, t].argmax()
        pitch = pitches[index, t]

        if pitch > 0:
            note = frequency_to_note(pitch)
            note_dict.setdefault(second, []).append(note)

    return note_dict

# 🎵 Convert Frequency to Western Notes
def frequency_to_note(freq):
    if freq == 0:
        return None
    return pretty_midi.note_number_to_name(int(librosa.hz_to_midi(freq)))

# 🎵 Western Notes to Swaras
note_to_swara = {
    "C": "Sa", "C#": "Re", "D": "Re", "D#": "Ga", "E": "Ga",
    "F": "Ma", "F#": "Ma#", "G": "Pa", "G#": "Dha", "A": "Dha",
    "A#": "Ni", "B": "Ni"
}

def convert_to_swara(notes):
    return [note_to_swara[note[:-1]] for note in notes if note and note[:-1] in note_to_swara]

# 🎵 Determine Chord per Second
def get_chord_for_second(notes):
    if not notes:
        return None
    most_common_notes = [note for note, _ in Counter(notes).most_common(3)]
    return determine_chord(most_common_notes)

# 🎵 Chord Mappings
chord_mappings = {
    frozenset(["Sa", "Ga", "Pa"]): "C Major", frozenset(["Sa", "Ma", "Pa"]): "C Major",
    frozenset(["Sa", "Re", "Pa"]): "D Major", frozenset(["Re", "Ma", "Dha"]): "D Major",
    frozenset(["Pa", "Dha", "Sa"]): "G Major", frozenset(["Pa", "Ni", "Sa"]): "G Major",
    frozenset(["Re", "Ga", "Dha"]): "A Major", frozenset(["Pa", "Dha", "Re"]): "B Major",

    frozenset(["Sa", "Ga", "Ni"]): "C Minor", frozenset(["Re", "Ga", "Pa"]): "D Minor",
    frozenset(["Re", "Ga", "Ni"]): "A Minor", frozenset(["Re", "Dha", "Ni"]): "D Minor",
    frozenset(["Ga", "Pa", "Ni"]): "E Minor", frozenset(["Sa", "Ma", "Ni"]): "F Major",
    frozenset(["Sa", "Dha", "Ni"]): "F Minor", frozenset(["Ma", "Dha", "Ni"]): "F Major",
    frozenset(["Ma", "Pa", "Dha"]): "B Minor",

    frozenset(["Sa", "Ga", "Dha"]): "C Major", frozenset(["Re", "Ga#", "Pa"]): "D Major",
    frozenset(["Ga", "Pa#", "Ni"]): "E Major", frozenset(["Ma", "Dha#", "Ni"]): "F Major",
    frozenset(["Sa", "Re", "Ga"]): "C Minor", frozenset(["Re", "Ga", "Ma"]): "D Minor",
    frozenset(["Ga", "Pa", "Dha"]): "E Minor",

    frozenset(["Sa", "Re", "Pa"]): "C Major", frozenset(["Sa", "Ma", "Pa"]): "C Major",
    frozenset(["Re", "Ga", "Dha"]): "D Major", frozenset(["Sa", "Pa"]): "C Major",
    frozenset(["Re", "A"]): "D Minor", frozenset(["Ga", "Ni"]): "E Minor",
    frozenset(["Ma", "Dha"]): "F Major",
}

# 🎵 Map Notes to Chords
def determine_chord(notes):
    return chord_mappings.get(frozenset(notes), "Unknown Chord")

# 🎵 Main Execution
def main(mp3_file):
    wav_file = convert_mp3_to_wav(mp3_file)
    pitch_data = extract_pitches(wav_file)

    chords_per_second = {
        second: get_chord_for_second(convert_to_swara(notes))
        for second, notes in pitch_data.items()
    }

    for second, chord in sorted(chords_per_second.items()):
        print(f"{second}: {chord}")

    return chords_per_second

# 🎵 Run Program
mp3_filename = "Sample.mp3"  # Change to your file
chords_per_second = main(mp3_filename)


0: G Major
1: G Major
2: G Major
3: Unknown Chord
4: B Major
5: D Minor
6: Unknown Chord
7: F Minor
8: Unknown Chord
9: E Minor
10: Unknown Chord
11: B Major
12: B Major
13: C Major
14: B Major
15: D Minor
16: A Minor
17: Unknown Chord
18: D Major
19: B Major
20: B Major
21: B Major
22: D Minor
23: E Minor
24: Unknown Chord
25: E Minor
26: Unknown Chord
27: Unknown Chord
28: Unknown Chord
29: B Major
30: F Minor
31: D Minor
32: A Minor
33: A Minor
34: Unknown Chord
35: F Minor
36: F Minor
37: Unknown Chord
38: Unknown Chord
39: C Minor
40: Unknown Chord
41: G Major
42: Unknown Chord
43: D Major
44: D Major
45: Unknown Chord
46: Unknown Chord
47: D Minor
48: A Minor
49: Unknown Chord
50: F Minor
51: D Minor
52: D Minor
53: F Minor
54: Unknown Chord
55: Unknown Chord
56: F Minor
57: B Major
58: Unknown Chord
59: D Minor
60: Unknown Chord
61: Unknown Chord
62: D Minor
63: A Minor
64: Unknown Chord
65: Unknown Chord
66: Unknown Chord
67: Unknown Chord
68: G Major
69: Unknown Chord
70: A Mi

In [12]:
import pandas as pd
df=pd.DataFrame(chords_per_second.items(),columns=['Second','Chord'])
df['Chord'].unique()

array(['G Major', 'Unknown Chord', 'B Major', 'D Minor', 'F Minor',
       'E Minor', 'C Major', 'A Minor', 'D Major', 'C Minor', 'B Minor',
       'F Major'], dtype=object)